In [2]:
import sys
from pathlib import Path

import torch
import numpy as np
from tqdm import tqdm

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Add the parent directory of 'notebooks' to sys.path
sys.path.append(str(Path('.').resolve().parent))

# Now you can import the function
from posenc.enums import PatchEmbeddingType, PosEncType
from posenc.modules.mnistmodel import MNISTModel
from posenc.datasets.medmnist import MNISTDataModule

from medmnist import INFO, Evaluator
from medmnist.evaluator import getAUC, getACC
from medmnist import VesselMNIST3D, AdrenalMNIST3D

from lightning.pytorch import Trainer, seed_everything
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.callbacks import ModelCheckpoint

from torchmetrics.classification import MulticlassConfusionMatrix, MulticlassAUROC, MulticlassAccuracy

sns.set_theme(context="paper", style="whitegrid", font_scale=1.5)

# # Default used by PyTorch
# torch.set_float32_matmul_precision("highest")
# Faster, but less precise
torch.set_float32_matmul_precision("high")
# # Even faster, but also less precise
# torch.set_float32_matmul_precision("medium")

seed_everything(42)

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
[rank: 0] Seed set to 42


42

In [3]:
base = Path("/sc-scratch/sc-scratch-gbm-radiomics/posenc/mnist")

MNISTROOT = "/sc-scratch/sc-scratch-gbm-radiomics/medmnist"
FLAG = "organmnist3d".lower()
info = INFO[FLAG]
task = info["task"]

In [4]:
def get_organmnist_checkpoints():
    organ_checkpoint = base / "organmnist3d" / "axial"
    checkpoints = [str(x) for x in organ_checkpoint.rglob("*.ckpt")]
    checkpoints = checkpoints + [
        '/sc-scratch/sc-scratch-gbm-radiomics/posenc/mnist/organmnist3d/non-square-shapes/ISOFPE-reduced/anisotropy_1_1_1/epoch=35-step=4392.ckpt',
        '/sc-scratch/sc-scratch-gbm-radiomics/posenc/mnist/organmnist3d/non-square-shapes/ISOFPE-reduced/anisotropy_1_1_4/epoch=36-step=4514.ckpt',    
        '/sc-scratch/sc-scratch-gbm-radiomics/posenc/mnist/organmnist3d/non-square-shapes/SINCOS/anisotropy_1_1_1/epoch=44-step=5490.ckpt',
        '/sc-scratch/sc-scratch-gbm-radiomics/posenc/mnist/organmnist3d/non-square-shapes/SINCOS/anisotropy_1_1_4/epoch=42-step=5246.ckpt',
    ]
    return sorted(checkpoints)

checkpoints = get_organmnist_checkpoints()

In [6]:
test_result = []
for checkpoint in tqdm(checkpoints):
    checkpoint = Path(checkpoint)
    method = checkpoint.parents[1].name
    anisotropy = [int(x) for x in checkpoint.parent.name.split("_")[1:]]

    dm = MNISTDataModule(MNISTROOT, FLAG, anisotropy=anisotropy, interpolate=False)
    dm.setup()

    image_size = torch.tensor([64, 64, 64])
    image_size = (image_size - 1) // torch.tensor(anisotropy) + 1

    image_patch_size = torch.round(torch.tensor([4, 4, 4]) / torch.tensor(anisotropy)).clip(1).type(torch.int)
    # image_patch_size = torch.round(torch.tensor([2, 2, 2]) / torch.tensor(anisotropy)).clip(1).type(torch.int)
    
    loader = dm.test_dataloader()
    model = MNISTModel.load_from_checkpoint(checkpoint, image_patch_size=image_patch_size.tolist(), image_size=image_size.tolist(), strict=False)
    model.eval()
    model.cuda()

    y_true = []
    y_score = []
    
    with torch.no_grad():
        for batch in loader:
            x, y = batch
            x = x.cuda()
            logits = model(x)
            probs = model.activation(logits)

            y_true.append(y)
            y_score.append(probs.cpu())

    test_result.append((method, anisotropy, torch.cat(y_true).numpy(), torch.cat(y_score).numpy()))

100%|██████████| 42/42 [05:45<00:00,  8.22s/it]


In [7]:
def get_stratified_bootstrap_indices(y_true):
    """
    Generates a set of indices for a single stratified bootstrap sample,
    preserving the original class distribution.

    Args:
        y_true (np.array): An array-like object containing the true labels.
                           Can be 1D (for binary/multi-class labels) or
                           2D (for one-hot encoded labels, where argmax will be used).

    Returns:
        np.array: A 1D numpy array of indices for the bootstrap sample.
                  These indices are shuffled.
    """
    if y_true.ndim > 1 and y_true.shape[1] > 1:
        # Assume y_true is one-hot encoded, convert to class labels
        true_labels = np.argmax(y_true, axis=1)
    else:
        # y_true is already in label format (binary or multi-class labels)
        true_labels = y_true

    bootstrapped_indices = []
    unique_classes = np.unique(true_labels)

    for class_label in unique_classes:
        # Get all original indices for the current class
        class_indices = np.where(true_labels == class_label)[0]
        # Number of samples for this class in the original dataset
        n_samples_in_class = len(class_indices)
        # Sample with replacement from these class-specific indices
        bootstrapped_class_indices = np.random.choice(
            class_indices,
            size=n_samples_in_class,
            replace=True
        )
        bootstrapped_indices.extend(bootstrapped_class_indices)

    # Convert to numpy array and shuffle to mix samples from different classes
    final_indices = np.array(bootstrapped_indices)
    np.random.shuffle(final_indices)

    return final_indices


In [49]:
results = []
n_classes = 11
for method, anisotropy, y_true, y_score in tqdm(test_result):
    for i in range(100):
        if info["task"] == "binary-class":
            indeces = get_stratified_bootstrap_indices(y_true)
        else:
            indeces = np.random.choice(np.arange(y_true.shape[0]), size=y_true.shape[0])

        acc = getACC(y_true[indeces, ...], y_score[indeces, ...], info["task"])
        auc = getAUC(y_true[indeces, ...], y_score[indeces, ...], info["task"])
        
        result_dict = {
            'method': method,
            'anisotropy': '_'.join(map(str, anisotropy)),
            'acc': acc,
            'auc': auc,
            'sample': i
        }
        results.append(result_dict)

# Convert to DataFrame
df = pd.DataFrame(results)

100%|██████████| 42/42 [00:46<00:00,  1.12s/it]


In [50]:
df.method = df.method.replace({"ISOFPE": "AFPE", "LEARNABLE": "Learnable", "NONE": "None", "SINCOS": "Sincos", "small": "AFPE*"})
col_name_translate = {
    "acc": "Accuracy",
    "auc": "AUROC",
}
df.rename(columns=col_name_translate, inplace=True)
df.anisotropy = df.anisotropy.str.replace("_", " ")
df.loc[(df.method == "ISOFPE-reduced"), "method"] = "AFPE"

In [51]:
auroc_results = df.groupby(["method", "anisotropy"], as_index=False).AUROC.agg(["mean", "std"]).round(3)

for p in ["None", "Learnable", "Sincos", "FOURIER", "LFPE", "AFPE"]:
    r = auroc_results[(auroc_results.method == p) & auroc_results.anisotropy.isin(["1 1 1", "1 1 4", "1 1 6", "1 1 8"])]

    for i, row in r.iterrows():
        print(f"{row.method} {row.anisotropy} \\pmnum{{{row['mean']}}}{{{row['std']}}}")
    print()

None 1 1 1 \pmnum{0.992}{0.001}
None 1 1 4 \pmnum{0.968}{0.003}
None 1 1 6 \pmnum{0.964}{0.003}
None 1 1 8 \pmnum{0.959}{0.004}

Learnable 1 1 1 \pmnum{0.983}{0.002}
Learnable 1 1 4 \pmnum{0.953}{0.004}
Learnable 1 1 6 \pmnum{0.951}{0.004}
Learnable 1 1 8 \pmnum{0.945}{0.004}

Sincos 1 1 1 \pmnum{0.995}{0.001}
Sincos 1 1 4 \pmnum{0.98}{0.002}
Sincos 1 1 6 \pmnum{0.976}{0.003}
Sincos 1 1 8 \pmnum{0.969}{0.003}

FOURIER 1 1 1 \pmnum{0.994}{0.001}
FOURIER 1 1 4 \pmnum{0.979}{0.002}
FOURIER 1 1 6 \pmnum{0.975}{0.003}
FOURIER 1 1 8 \pmnum{0.958}{0.003}

LFPE 1 1 1 \pmnum{0.993}{0.001}
LFPE 1 1 4 \pmnum{0.968}{0.003}
LFPE 1 1 6 \pmnum{0.972}{0.003}
LFPE 1 1 8 \pmnum{0.969}{0.003}

AFPE 1 1 1 \pmnum{0.994}{0.001}
AFPE 1 1 4 \pmnum{0.984}{0.002}
AFPE 1 1 6 \pmnum{0.983}{0.002}
AFPE 1 1 8 \pmnum{0.972}{0.003}



In [66]:
# df.to_csv("organmnist_result_bootsrapped.csv")

In [69]:
auroc_results = df.groupby(["method", "anisotropy"], as_index=False).Accuracy.agg(["mean", "std"]).round(3)

for p in ["None", "Learnable", "Sincos", "FOURIER", "LFPE", "AFPE"]:
    r = auroc_results[(auroc_results.method == p) & auroc_results.anisotropy.isin(["1 1 1", "1 1 4", "1 1 6", "1 1 8"])]

    for i, row in r.iterrows():
        print(f"{row.method} {row.anisotropy} \\pmnum{{{row['mean']}}}{{{row['std']}}}")
    print()

None 1 1 1 \pmnum{0.762}{0.02}
None 1 1 4 \pmnum{0.85}{0.019}
None 1 1 6 \pmnum{0.768}{0.0}
None 1 1 8 \pmnum{0.816}{0.02}

Learnable 1 1 1 \pmnum{0.768}{0.0}
Learnable 1 1 4 \pmnum{0.822}{0.015}
Learnable 1 1 6 \pmnum{0.738}{0.016}
Learnable 1 1 8 \pmnum{0.792}{0.023}

Sincos 1 1 1 \pmnum{0.833}{0.019}
Sincos 1 1 4 \pmnum{0.809}{0.019}
Sincos 1 1 6 \pmnum{0.814}{0.018}
Sincos 1 1 8 \pmnum{0.824}{0.021}

FOURIER 1 1 1 \pmnum{0.819}{0.019}
FOURIER 1 1 4 \pmnum{0.799}{0.021}
FOURIER 1 1 6 \pmnum{0.768}{0.018}
FOURIER 1 1 8 \pmnum{0.807}{0.013}

LFPE 1 1 1 \pmnum{0.764}{0.022}
LFPE 1 1 4 \pmnum{0.778}{0.01}
LFPE 1 1 6 \pmnum{0.827}{0.019}
LFPE 1 1 8 \pmnum{0.797}{0.021}

AFPE 1 1 1 \pmnum{0.798}{0.018}
AFPE 1 1 4 \pmnum{0.809}{0.016}
AFPE 1 1 6 \pmnum{0.821}{0.017}
AFPE 1 1 8 \pmnum{0.772}{0.013}



In [48]:
from collections import defaultdict
from scipy.stats import ttest_rel

stat_test_predictions = defaultdict(list)
for method, anisotropy, y_true, y_score in test_result:
    if method in ["ISOFPE-reduced", "ISOFPE", "SINCOS"]:
        stat_test_predictions[str(anisotropy)].append((method, y_true, y_score))

for anisotropy, (a, b) in stat_test_predictions.items():
    
    method_a, y_true_a, y_score_a = a
    method_b, y_true_b, y_score_b = b

    auc_a = []
    auc_b = []
    
    for _ in range(50):
        indeces = np.random.choice(np.arange(y_true_a.shape[0]), size=y_true_a.shape[0])
        auc_a_ = getAUC(y_true_a[indeces, ...], y_score_a[indeces, ...], info["task"])

        # indeces = np.random.choice(np.arange(y_true_b.shape[0]), size=y_true_b.shape[0])
        auc_b_ = getAUC(y_true_b[indeces, ...], y_score_b[indeces, ...], info["task"])

        auc_a.append(auc_a_)
        auc_b.append(auc_b_)

    statistic = ttest_rel(auc_a, auc_b)
    print(anisotropy, statistic.pvalue < 0.05, round(statistic.pvalue, 5))
    print()

[1, 1, 2] True 0.0


[1, 1, 3] True 0.0


[1, 1, 5] True 0.0


[1, 1, 6] True 0.0


[1, 1, 8] True 0.0


[1, 1, 1] True 0.00023


[1, 1, 4] True 0.0


